# SimRAG Model Testing on Google Colab

This notebook tests **any fine-tuned SimRAG model** (Stage 1, Stage 2, or any checkpoint) using Google Colab's free T4 GPU (15GB VRAM).

## 📋 Complete Workflow

**Step 1: Prepare Files Locally**
```bash
# Export your trained model
simrag experiment export

# Create documents ZIP
cd data
zip -r documents.zip documents/  # Linux/Mac
# OR
Compress-Archive -Path documents -DestinationPath documents.zip  # Windows
```

**Step 2: Run This Notebook**
1. Upload `documents.zip` (when prompted)
2. Notebook automatically runs baseline tests (base model, no fine-tuning)
3. Upload your model ZIP (e.g., `checkpoint-1000-fixed.zip`)
4. Notebook automatically runs fine-tuned model tests
5. Automatic comparison and download

**Step 3: View Results Locally**
1. Download the `comparison_results_*.json` file from Colab
2. Place it in: `comparison_results/` (project root)
3. Run: `simrag experiment results` to view formatted results

## ✅ Fair Comparison Guarantees

Both baseline and fine-tuned models are tested with **identical conditions**:
- **Same embedding model**: `sentence-transformers/all-MiniLM-L6-v2` (384 dimensions)
- **Same retrieval**: top_k=5 documents per question
- **Same prompt format**: `Context:\n{context}\n\nQuestion: {question}\n\nAnswer:`
- **Same test questions**: 30 standard questions from `get_test_questions()`
- **Same evaluation metrics**: `evaluate_answer_quality()` (length, context relevance, question relevance, refusal detection)
- **Same generation params**: temperature=0.7, max_tokens=512, top_p=0.9
- **Same documents**: Identical document set indexed in ChromaDB

## 📊 Evaluation Metrics

**Context Score**: Average similarity between retrieved documents and query (0.0-1.0)
- Higher = better document retrieval

**Answer Quality Score**: Multi-metric evaluation (0.0-1.0)
- **Length**: Reasonable answer length (20-500 chars = 1.0)
- **Context Relevance**: Word overlap between answer and retrieved context
- **Question Relevance**: Word overlap between answer and question
- **Not Refusal**: Answer attempts to respond (not "I don't know")
- **Overall**: Average of all quality metrics

**Statistical Analysis**: 
- Mean ± Standard Deviation
- 95% Confidence Intervals
- Statistical significance test (CI overlap)

## 🎯 What Models Can You Test?

- ✅ **Any Stage 1 model** (instruction following)
- ✅ **Any Stage 2 model** (domain adaptation)
- ✅ **Any checkpoint** (checkpoint-500, checkpoint-1000, etc.)
- ✅ **Any model version** (v1.0, v1.8, etc.)

The notebook automatically detects model metadata from `adapter_config.json`.


In [ ]:
# Install dependencies
!pip install -q transformers torch peft accelerate bitsandbytes chromadb sentence-transformers beautifulsoup4

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import json
from pathlib import Path
import time
from datetime import datetime
import re
from html.parser import HTMLParser

# Check GPU
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

## Utility Functions (from simrag_reproduction/experiments/utils.py)

These match your local testing infrastructure exactly:

In [ ]:
class HTMLTextExtractor(HTMLParser):
    """Extract text content from HTML files (exact copy from utils.py)"""
    
    def __init__(self):
        super().__init__()
        self.text = []
        self.skip_tags = {'script', 'style', 'meta', 'link', 'head'}
        self.in_skip_tag = False
    
    def handle_starttag(self, tag, attrs):
        if tag.lower() in self.skip_tags:
            self.in_skip_tag = True
    
    def handle_endtag(self, tag):
        if tag.lower() in self.skip_tags:
            self.in_skip_tag = False
        elif tag.lower() in {'p', 'div', 'br', 'li'}:
            self.text.append('\n')
    
    def handle_data(self, data):
        if not self.in_skip_tag:
            cleaned = data.strip()
            if cleaned:
                self.text.append(cleaned)
    
    def get_text(self):
        text = ' '.join(self.text)
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'\n\s*\n', '\n\n', text)
        return text.strip()


def extract_text_from_html(html_path):
    """Extract text from HTML file"""
    try:
        with open(html_path, 'r', encoding='utf-8', errors='ignore') as f:
            html_content = f.read()
        parser = HTMLTextExtractor()
        parser.feed(html_content)
        return parser.get_text()
    except Exception as e:
        print(f"Error extracting text from {html_path}: {e}")
        return ""


def load_documents_from_folder(folder_path, include_html=True):
    """Load documents from folder (exact copy from utils.py)"""
    folder = Path(folder_path).resolve()
    if not folder.exists():
        raise ValueError(f"Folder not found: {folder_path}")
    
    documents = []
    
    # Load .txt and .md files
    for ext in ['.txt', '.md']:
        for file_path in folder.glob(f"**/*{ext}"):
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    content = f.read().strip()
                    if content:
                        documents.append(content)
            except Exception as e:
                print(f"Error reading {file_path}: {e}")
    
    # Load HTML files
    if include_html:
        for html_path in folder.glob("**/*.html"):
            text = extract_text_from_html(html_path)
            if text:
                documents.append(text)
    
    return documents


def get_test_questions():
    """Standard test questions (exact copy from utils.py) - Expanded to 30 questions"""
    return [
        # Original questions (1-10)
        "What is Docker?",
        "How does CI/CD work?",
        "What is DevOps?",
        "How do you containerize an application?",
        "What is the difference between Docker and virtual machines?",
        "How does Google Cloud Platform work?",
        "What are the benefits of using containers?",
        "Explain the Python standard library.",
        "What is RAG in generative AI?",
        "How do you build a Docker image?",
        
        # Docker - Technical depth (11-18)
        "What are Docker layers and how do they optimize image builds?",
        "How do Docker volumes work and when should you use them?",
        "What is the difference between Docker COPY and ADD commands?",
        "How does Docker networking work between containers?",
        "What are Docker multi-stage builds and why are they useful?",
        "How do you optimize a Dockerfile for smaller image sizes?",
        "What is the purpose of the Docker EXPOSE instruction?",
        "How do you handle environment variables in Docker containers?",
        
        # CI/CD - Automation and pipelines (19-23)
        "What are the key stages in a CI/CD pipeline?",
        "How do you implement automated testing in CI/CD?",
        "What is the difference between continuous integration and continuous deployment?",
        "How do you handle secrets and credentials in CI/CD pipelines?",
        "What are best practices for CI/CD pipeline design?",
        
        # DevOps - Practices and methodologies (24-26)
        "What are the core principles of DevOps?",
        "How does infrastructure as code relate to DevOps?",
        "What is the role of monitoring and logging in DevOps?",
        
        # Google Cloud Platform (27-28)
        "What are the main services offered by Google Cloud Platform?",
        "How do you deploy applications to Google Cloud Platform?",
        
        # Python - Advanced topics (29-30)
        "What are the key modules in the Python standard library?",
        "How do you use Python's built-in data structures effectively?"
    ]


def evaluate_answer_quality(question, answer, context):
    """
    Evaluate answer quality using rule-based metrics (exact copy from utils.py)
    
    This function provides automated, objective scoring for fair comparison:
    - Length: Ensures answers are reasonable length (not too short/long)
    - Context Relevance: Measures word overlap with retrieved context
    - Question Relevance: Measures word overlap with question
    - Not Refusal: Detects if model refuses to answer
    
    Note: This is rule-based (not semantic), but provides consistent,
    objective metrics for comparing baseline vs fine-tuned models.
    Both models are evaluated with identical criteria.
    """
    scores = {}
    
    # 1. Length scoring
    answer_len = len(answer.strip())
    if answer_len == 0:
        scores['length_score'] = 0.0
    elif answer_len < 20:
        scores['length_score'] = 0.5
    elif answer_len > 500:
        scores['length_score'] = 0.7
    else:
        scores['length_score'] = 1.0
    
    # 2. Context relevance
    if context:
        context_words = set(w.lower() for w in context.split() if len(w) > 3)
        answer_words = set(w.lower() for w in answer.split() if len(w) > 3)
        if len(context_words) > 0 and len(answer_words) > 0:
            overlap = len(context_words & answer_words)
            scores['context_relevance'] = min(1.0, overlap / max(len(answer_words), 5))
        else:
            scores['context_relevance'] = 0.0
    else:
        scores['context_relevance'] = 0.0
    
    # 3. Not a refusal
    refusal_phrases = [
        "don't know", "do not know", "cannot answer", "can't answer",
        "no information", "not sure", "unclear", "i'm sorry",
        "i apologize", "unable to"
    ]
    answer_lower = answer.lower()
    is_refusal = any(phrase in answer_lower for phrase in refusal_phrases)
    scores['not_refusal'] = 0.0 if is_refusal else 1.0
    
    # 4. Question relevance
    question_words = set(w.lower() for w in question.split() if len(w) > 3)
    answer_words = set(w.lower() for w in answer.split() if len(w) > 3)
    if len(question_words) > 0 and len(answer_words) > 0:
        overlap = len(question_words & answer_words)
        scores['question_relevance'] = min(1.0, overlap / max(len(question_words), 3))
    else:
        scores['question_relevance'] = 0.0
    
    # 5. Overall score
    metric_values = [
        scores['length_score'],
        scores['context_relevance'],
        scores['not_refusal'],
        scores['question_relevance']
    ]
    scores['overall_score'] = sum(metric_values) / len(metric_values)
    
    return scores

print("✓ Utility functions loaded")

## 📤 Step 1: Upload Your Fine-Tuned Model

**How to Create Model ZIP Locally:**
```bash
# Option 1: Use the export command (recommended)
simrag experiment export
# Follow the interactive menu to select your model

# Option 2: Manual ZIP creation
cd tuned_models/model_1b/stage_1/v1.0  # Adjust path to your model
# Windows PowerShell:
Compress-Archive -Path checkpoint-1000 -DestinationPath checkpoint-1000-fixed.zip
# Linux/Mac:
zip -r checkpoint-1000-fixed.zip checkpoint-1000/
```

**What the ZIP Should Contain:**
- ✅ `adapter_model.safetensors` (required - LoRA adapter weights)
- ✅ `adapter_config.json` (required - model metadata)
- ⚠️ Other files are optional but recommended

**Supported Model Types:**
- Any checkpoint directory (e.g., `checkpoint-500`, `checkpoint-1000`)
- Any version directory (e.g., `v1.8`, `v2.0`)
- Stage 1 or Stage 2 models
- The notebook will automatically detect model type and metadata


In [ ]:
from google.colab import files
import zipfile
import os

def find_adapter_dirs(root_path='.'):
    """
    Recursively find directories containing adapter_model.safetensors and adapter_config.json
    Handles both flat ZIPs (files at root) and nested ZIPs (files in subdirectories)
    """
    adapter_dirs = []
    root = Path(root_path)
    
    # First, check if files are at the root level (flat ZIP)
    adapter_file = root / 'adapter_model.safetensors'
    config_file = root / 'adapter_config.json'
    if adapter_file.exists() and config_file.exists():
        adapter_dirs.append(str(root))
        print(f"  Found at root: {root}")
    
    # Then recursively search subdirectories
    for path in root.rglob('adapter_model.safetensors'):
        dir_path = path.parent
        config_path = dir_path / 'adapter_config.json'
        if config_path.exists():
            rel_path = dir_path.relative_to(root)
            if str(rel_path) not in [str(Path(d).relative_to(root)) if Path(d).is_absolute() else d for d in adapter_dirs]:
                adapter_dirs.append(str(dir_path))
                print(f"  Found in subdirectory: {rel_path}")
    
    return adapter_dirs

# Upload checkpoint ZIP
print("Upload your checkpoint ZIP file:")
uploaded = files.upload()

# Extract and find checkpoint files
checkpoint_dir = None
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        print(f"\nExtracting {filename}...")
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('.')

        # Find adapter_model.safetensors by recursively searching filesystem
        print("\n🔍 Looking for adapter_model.safetensors...")
        
        # Recursively search for adapter directories
        adapter_dirs = find_adapter_dirs('.')
        
        if adapter_dirs:
            # Use the first found directory (prefer root if both exist)
            # Sort to prefer shorter paths (root before nested)
            adapter_dirs.sort(key=lambda x: len(str(x)))
            checkpoint_dir = adapter_dirs[0]
            print(f"\n✓ Found checkpoint at: {checkpoint_dir}")

            # Verify and show file sizes
            checkpoint_path = Path(checkpoint_dir)
            required_files = ['adapter_model.safetensors', 'adapter_config.json']
            
            for req_file in required_files:
                file_path = checkpoint_path / req_file
                if file_path.exists():
                    size_mb = file_path.stat().st_size / (1024 * 1024)
                    print(f"  ✓ {req_file} ({size_mb:.1f} MB)")
                else:
                    print(f"  ✗ {req_file} MISSING!")
            
            break
        else:
            print("❌ No adapter files found in ZIP!")
            print("\nExtracted structure:")
            for item in sorted(Path('.').iterdir()):
                if item.is_dir() and not item.name.startswith('.'):
                    print(f"  - {item.name}/")
                    # Show first few files in each directory
                    try:
                        for subitem in list(item.iterdir())[:5]:
                            marker = "/" if subitem.is_dir() else ""
                            print(f"    {subitem.name}{marker}")
                    except:
                        pass
            raise Exception("Required adapter files (adapter_model.safetensors, adapter_config.json) not found in ZIP!")

if not checkpoint_dir:
    raise Exception("No ZIP file uploaded or no adapter files found!")

# Ensure checkpoint_dir is a string path (not Path object)
checkpoint_dir = str(checkpoint_dir)
print(f"\n✅ Checkpoint ready: {checkpoint_dir}")

## ⚙️ Model Configuration

**Automatic Detection:**
The notebook automatically extracts model information from `adapter_config.json`:
- Base model name (e.g., `Qwen/Qwen2.5-1.5B-Instruct`)
- LoRA rank (r) and alpha parameters
- Model size (small/medium)

**Auto-Detection:**
The notebook attempts to detect stage and version from the checkpoint path. If detection fails, you can manually set:
- `MODEL_VERSION`: Your model version (e.g., "v1.0")
- `STAGE`: Training stage ("stage_1" or "stage_2")

In [ ]:
# Load adapter config to get base model info
adapter_path = Path(checkpoint_dir)
adapter_config_file = adapter_path / "adapter_config.json"

# Verify required files exist
assert (adapter_path / "adapter_model.safetensors").exists(), "adapter_model.safetensors not found!"
assert adapter_config_file.exists(), "adapter_config.json not found!"

# Load adapter config
with open(adapter_config_file, 'r') as f:
    adapter_config = json.load(f)

# Extract base model name
BASE_MODEL = adapter_config.get("base_model_name_or_path", "Qwen/Qwen2.5-1.5B-Instruct")
print(f"✓ Base model: {BASE_MODEL}")
print(f"✓ LoRA rank (r): {adapter_config.get('r', 'N/A')}")
print(f"✓ LoRA alpha: {adapter_config.get('lora_alpha', 'N/A')}")

# Auto-detect stage and version from path
ADAPTER_PATH = checkpoint_dir
checkpoint_path = Path(checkpoint_dir)

# Try to detect stage from path (look for "stage_1" or "stage_2")
path_str = str(checkpoint_path).lower()
if "stage_2" in path_str or "/stage2" in path_str:
    STAGE = "stage_2"
elif "stage_1" in path_str or "/stage1" in path_str:
    STAGE = "stage_1"
else:
    STAGE = "stage_1"  # Default fallback
    print(f"⚠️  Could not detect stage from path, defaulting to {STAGE}")

# Try to detect version from path (look for "v1.0", "v2.0", etc.)
version_match = None
for part in checkpoint_path.parts:
    if part.startswith("v") and len(part) > 1 and part[1:].replace(".", "").isdigit():
        version_match = part
        break

if version_match:
    MODEL_VERSION = version_match
else:
    # Fallback: try to extract from checkpoint_dir name
    dir_name = checkpoint_path.name
    if dir_name.startswith("v"):
        MODEL_VERSION = dir_name
    else:
        MODEL_VERSION = "unknown"  # User should update manually
        print(f"⚠️  Could not detect version from path, set MODEL_VERSION manually")

CHECKPOINT_NAME = checkpoint_path.name
MODEL_SIZE = "small" if "1.5B" in BASE_MODEL or "1B" in BASE_MODEL else "medium"

print(f"\n✓ Configuration:")
print(f"  Model size: {MODEL_SIZE}")
print(f"  Stage: {STAGE}")
print(f"  Version: {MODEL_VERSION}")
print(f"  Checkpoint: {CHECKPOINT_NAME}")
if MODEL_VERSION == "unknown":
    print(f"\n⚠️  Please update MODEL_VERSION manually if needed")

## 🔧 Load Fine-Tuned Model

**Memory Optimization:**
- Uses 4-bit quantization (BitsAndBytes) to fit in Colab's 15GB VRAM
- Loads base model + LoRA adapters via PEFT
- Maintains quality while reducing memory usage by ~75%

**Expected Load Time:** 2-3 minutes

In [ ]:
print("Loading base model and adapters...")
print("This will take 2-3 minutes...")

# 4-bit quantization config (uses ~2GB VRAM for 1.5B model)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load LoRA adapters
model = PeftModel.from_pretrained(model, ADAPTER_PATH)
model.eval()

print("✓ Model loaded successfully!")
print(f"Model memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")

## 📄 Step 2: Upload Test Documents

**Create Documents ZIP Locally:**
```bash
cd data
# Windows PowerShell:
Compress-Archive -Path documents -DestinationPath documents.zip
# Linux/Mac:
zip -r documents.zip documents/
```

**What Documents Are Used:**
- All `.txt`, `.md`, and `.html` files from the documents folder
- Same documents used for training and baseline testing
- Indexed in ChromaDB for semantic search

**Note:** Upload documents **before** running baseline tests. The same document set is used for both baseline and fine-tuned model testing to ensure fair comparison.

In [ ]:
# Upload documents ZIP
print("Upload your documents ZIP file:")
uploaded = files.upload()

# Extract documents
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        print(f"Extracting {filename}...")
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('./documents')
        print(f"Documents extracted to: ./documents")
        break

## 🔍 Setup RAG System

**Critical for Fair Comparison:**
This RAG system uses **identical configuration** to your local baseline tests:
- **Embedding model**: `sentence-transformers/all-MiniLM-L6-v2` (384 dimensions)
  - Same as local: `retriever.py` uses this exact model
- **Retrieval**: top_k=5 documents per question
  - Matches `RAGConfig.top_k = 5` from your local config
- **Prompt format**: `Context:\n{context}\n\nQuestion: {question}\n\nAnswer:`
  - Exact match to `rag_setup.py` lines 147-152
- **Database**: In-memory ChromaDB (no persistence)
  - Matches `use_persistent=False` in local testing

**Why This Matters:**
Any differences in results between baseline and fine-tuned models are **only** due to the model's fine-tuning, not configuration differences.

In [ ]:
import chromadb
from chromadb.utils import embedding_functions

# RAG Configuration (matches config.py defaults)
TOP_K = 5  # Number of documents to retrieve (RAGConfig.top_k = 5)
TEMPERATURE = 0.7  # Generation temperature (RAGConfig.temperature = 0.7)
MAX_TOKENS = 512  # Max new tokens (generous for complete answers)

# Initialize ChromaDB (in-memory, matching your test setup)
print("Initializing ChromaDB...")
client = chromadb.Client()

# CRITICAL: Use exact same embedding model as baseline
# This matches retriever.py: SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embedding_fn = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

collection = client.create_collection(
    name="model_test",
    embedding_function=embedding_fn
)

print("✓ Using sentence-transformers/all-MiniLM-L6-v2 (384 dimensions)")
print(f"✓ Retrieval: top_k={TOP_K}")
print(f"✓ Generation: temperature={TEMPERATURE}, max_tokens={MAX_TOKENS}")

In [ ]:
# Load documents using the same function as local testing
docs_dir = Path("./documents")
print(f"Loading documents from {docs_dir}...")
documents = load_documents_from_folder(str(docs_dir), include_html=True)
print(f"Loaded {len(documents)} documents")

if not documents:
    raise ValueError("No documents found!")

# Add to ChromaDB
doc_ids = [f"doc_{i}" for i in range(len(documents))]
collection.add(
    documents=documents,
    ids=doc_ids
)
print("✓ Documents indexed in ChromaDB")

# Store documents and doc_ids for reuse in baseline and fine-tuned tests
DOCUMENTS = documents
DOC_IDS = doc_ids

## 📊 Step 3: Run Baseline Tests

**Purpose:** Establish baseline performance using the **base model** (no fine-tuning)

**What Happens:**
1. Loads base model (e.g., `Qwen/Qwen2.5-1.5B-Instruct`) with 4-bit quantization
2. Tests all 30 standard questions through the RAG pipeline
3. Records: answers, context scores, answer quality, response times
4. Cleans up model to free VRAM for fine-tuned model testing

**Expected Runtime:** ~10-15 minutes (depends on model size and question complexity)

**Note:** Baseline model is loaded **once** and reused for all questions (optimized for speed).

In [ ]:
# Get standard test questions (same as local testing)
# These 30 questions are used for both baseline and fine-tuned model testing
# to ensure fair comparison
TEST_QUESTIONS = get_test_questions()
print(f"Testing with {len(TEST_QUESTIONS)} questions")
print("Questions:", TEST_QUESTIONS)

def query_rag_baseline(question, baseline_model, baseline_tokenizer, top_k=TOP_K):
    """Query RAG system with base model (no fine-tuning)"""
    # Retrieve context
    results = collection.query(
        query_texts=[question],
        n_results=top_k
    )
    
    context_docs = results['documents'][0] if results['documents'] else []
    context_scores = results['distances'][0] if results['distances'] else []
    context_scores = [max(0.0, 1.0 - d) for d in context_scores]
    
    context_text = "\n\n".join(context_docs)
    
    # Build prompt - EXACT format from rag_setup.py
    prompt = f"""Context:
{context_text}

Question: {question}

Answer:"""
    
    # Generate answer using pre-loaded baseline model
    inputs = baseline_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048).to(baseline_model.device)
    
    with torch.no_grad():
        outputs = baseline_model.generate(
            **inputs,
            max_new_tokens=MAX_TOKENS,
            temperature=TEMPERATURE,
            top_p=0.9,
            do_sample=True,
            pad_token_id=baseline_tokenizer.pad_token_id
        )
    
    answer = baseline_tokenizer.decode(outputs[0], skip_special_tokens=True)
    if prompt in answer:
        answer = answer.replace(prompt, "").strip()
    else:
        answer_marker = "Answer:"
        if answer_marker in answer:
            answer = answer.split(answer_marker)[-1].strip()
    
    return answer, context_docs, context_scores

def query_rag(question, top_k=TOP_K):
    """Query RAG system (matches local implementation)"""
    # Retrieve context
    results = collection.query(
        query_texts=[question],
        n_results=top_k
    )
    
    context_docs = results['documents'][0] if results['documents'] else []
    context_scores = results['distances'][0] if results['distances'] else []
    # Convert distances to similarity scores (1 - distance for cosine)
    # ChromaDB returns L2 distances, but for comparison we normalize to [0,1]
    context_scores = [max(0.0, 1.0 - d) for d in context_scores]
    
    context_text = "\n\n".join(context_docs)
    
    # Build prompt - EXACT format from rag_setup.py lines 147-152
    prompt = f"""Context:
{context_text}

Question: {question}

Answer:"""
    
    # Generate answer with same parameters as baseline
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048).to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=MAX_TOKENS,
            temperature=TEMPERATURE,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id
        )
    
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract just the answer part (remove prompt)
    if prompt in answer:
        answer = answer.replace(prompt, "").strip()
    else:
        # Fallback: try to find where answer starts
        answer_marker = "Answer:"
        if answer_marker in answer:
            answer = answer.split(answer_marker)[-1].strip()
    
    return answer, context_docs, context_scores


# Run baseline tests
print("="*60)
print("BASELINE TESTS (Base Model)")
print("="*60)

# Load baseline model ONCE (optimization - don't reload for each question)
print("Loading baseline model (this will take 1-2 minutes)...")
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

baseline_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)
baseline_tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
if baseline_tokenizer.pad_token is None:
    baseline_tokenizer.pad_token = baseline_tokenizer.eos_token
baseline_model.eval()
print("✓ Baseline model loaded")

baseline_results = {
    "experiment_type": "baseline",
    "timestamp": datetime.now().isoformat(),
    "config": {
        "model_name": BASE_MODEL,
        "top_k": TOP_K,
        "temperature": TEMPERATURE,
        "max_tokens": MAX_TOKENS,
        "embedding_model": "sentence-transformers/all-MiniLM-L6-v2"
    },
    "dataset": {
        "documents_folder": str(docs_dir),
        "num_documents": len(DOCUMENTS)
    },
    "questions": [],
    "answers": [],
    "response_times": [],
    "context_scores": [],
    "context_docs": [],
    "answer_quality_scores": []
}

print(f"\nRunning baseline tests on {len(TEST_QUESTIONS)} questions...\n")

for i, question in enumerate(TEST_QUESTIONS, 1):
    print(f"[{i}/{len(TEST_QUESTIONS)}] {question[:60]}...")
    
    start_time = time.time()
    try:
        answer, context_docs, context_scores = query_rag_baseline(question, baseline_model, baseline_tokenizer)
        elapsed = time.time() - start_time
        
        context_text = "\n\n".join(context_docs)
        quality_scores = evaluate_answer_quality(question, answer, context_text)
        
        baseline_results["questions"].append(question)
        baseline_results["answers"].append(answer)
        baseline_results["response_times"].append(elapsed)
        baseline_results["context_scores"].append(context_scores)
        baseline_results["context_docs"].append(context_docs)
        baseline_results["answer_quality_scores"].append(quality_scores)
        
        avg_context = sum(context_scores) / len(context_scores) if context_scores else 0.0
        print(f"  ✓ Answer: {len(answer)} chars, Context: {avg_context:.3f}, Quality: {quality_scores['overall_score']:.3f}, Time: {elapsed:.1f}s")
        
    except Exception as e:
        print(f"  ✗ Error: {e}")
        baseline_results["questions"].append(question)
        baseline_results["answers"].append(f"ERROR: {str(e)}")
        baseline_results["response_times"].append(0.0)
        baseline_results["context_scores"].append([])
        baseline_results["context_docs"].append([])
        baseline_results["answer_quality_scores"].append({
            "length_score": 0.0, "context_relevance": 0.0, "not_refusal": 0.0,
            "question_relevance": 0.0, "overall_score": 0.0
        })

# Calculate baseline summary
all_baseline_scores = [score for scores in baseline_results["context_scores"] if scores for score in scores]
baseline_quality_metrics = {}
if baseline_results["answer_quality_scores"]:
    metric_keys = ["length_score", "context_relevance", "not_refusal", "question_relevance", "overall_score"]
    for key in metric_keys:
        values = [q[key] for q in baseline_results["answer_quality_scores"] if isinstance(q, dict)]
        baseline_quality_metrics[f"avg_{key}"] = sum(values) / len(values) if values else 0.0

baseline_results["summary"] = {
    "avg_context_score": sum(all_baseline_scores) / len(all_baseline_scores) if all_baseline_scores else 0.0,
    "avg_response_time": sum(baseline_results["response_times"]) / len(baseline_results["response_times"]) if baseline_results["response_times"] else 0.0,
    "total_questions": len(TEST_QUESTIONS),
    "successful_queries": len([a for a in baseline_results["answers"] if not a.startswith("ERROR")]),
    **baseline_quality_metrics
}

print("\n=== Baseline Tests Complete ===")
print(f"Avg context score: {baseline_results['summary']['avg_context_score']:.3f}")
print(f"Avg answer quality: {baseline_results['summary'].get('avg_overall_score', 0.0):.3f}")
print(f"Avg response time: {baseline_results['summary']['avg_response_time']:.2f}s")

# Clean up baseline model to free VRAM for fine-tuned model
print("\nCleaning up baseline model...")
del baseline_model
del baseline_tokenizer
torch.cuda.empty_cache()
print("✓ Baseline model unloaded")

# Now run fine-tuned model tests
# This uses the SAME test questions, documents, and evaluation metrics
# as the baseline to ensure fair comparison
print("\n" + "="*60)
print("FINE-TUNED MODEL TESTS")
print("="*60)
print("Testing fine-tuned model with identical conditions as baseline...")

# Initialize results structure (matches test_model.py format)
results = {
    "experiment_type": "model_test",
    "timestamp": datetime.now().isoformat(),
    "model": {
        "path": ADAPTER_PATH,
        "stage": STAGE,
        "version": MODEL_VERSION,
        "checkpoint": CHECKPOINT_NAME,
        "provider": "colab_transformers",
        "base_model": BASE_MODEL,
        "model_size": MODEL_SIZE,
        "lora_r": adapter_config.get('r'),
        "lora_alpha": adapter_config.get('lora_alpha')
    },
    "config": {
        "top_k": TOP_K,
        "temperature": TEMPERATURE,
        "max_tokens": MAX_TOKENS,
        "embedding_model": "sentence-transformers/all-MiniLM-L6-v2"
    },
    "dataset": {
        "documents_folder": str(docs_dir),
        "num_documents": len(DOCUMENTS)
    },
    "questions": [],
    "answers": [],
    "response_times": [],
    "context_scores": [],
    "context_docs": [],
    "answer_quality_scores": []
}

print(f"\nRunning tests on {len(TEST_QUESTIONS)} questions...\n")

for i, question in enumerate(TEST_QUESTIONS, 1):
    print(f"[{i}/{len(TEST_QUESTIONS)}] {question[:60]}...")
    
    start_time = time.time()
    try:
        answer, context_docs, context_scores = query_rag(question)
        elapsed = time.time() - start_time
        
        # Evaluate answer quality (same as local)
        context_text = "\n\n".join(context_docs)
        quality_scores = evaluate_answer_quality(question, answer, context_text)
        
        results["questions"].append(question)
        results["answers"].append(answer)
        results["response_times"].append(elapsed)
        results["context_scores"].append(context_scores)
        results["context_docs"].append(context_docs)
        results["answer_quality_scores"].append(quality_scores)
        
        avg_context = sum(context_scores) / len(context_scores) if context_scores else 0.0
        print(f"  ✓ Answer: {len(answer)} chars, Context: {avg_context:.3f}, Quality: {quality_scores['overall_score']:.3f}, Time: {elapsed:.1f}s")
        print(f"    Preview: {answer[:80]}...")
        
    except Exception as e:
        print(f"  ✗ Error: {e}")
        import traceback
        traceback.print_exc()
        
        results["questions"].append(question)
        results["answers"].append(f"ERROR: {str(e)}")
        results["response_times"].append(0.0)
        results["context_scores"].append([])
        results["context_docs"].append([])
        results["answer_quality_scores"].append({
            "length_score": 0.0,
            "context_relevance": 0.0,
            "not_refusal": 0.0,
            "question_relevance": 0.0,
            "overall_score": 0.0
        })

# Calculate summary (matches local format)
all_context_scores = [score for scores in results["context_scores"] if scores for score in scores]

# Calculate quality metrics
quality_metrics = {}
if results["answer_quality_scores"]:
    metric_keys = ["length_score", "context_relevance", "not_refusal", "question_relevance", "overall_score"]
    for key in metric_keys:
        values = [q[key] for q in results["answer_quality_scores"] if isinstance(q, dict)]
        quality_metrics[f"avg_{key}"] = sum(values) / len(values) if values else 0.0

results["summary"] = {
    "avg_context_score": sum(all_context_scores) / len(all_context_scores) if all_context_scores else 0.0,
    "avg_response_time": sum(results["response_times"]) / len(results["response_times"]) if results["response_times"] else 0.0,
    "total_questions": len(TEST_QUESTIONS),
    "successful_queries": len([a for a in results["answers"] if not a.startswith("ERROR")]),
    **quality_metrics
}

print("\n=== Fine-Tuned Model Tests Complete ===")
print(f"Avg context score: {results['summary']['avg_context_score']:.3f}")
print(f"Avg answer quality: {results['summary'].get('avg_overall_score', 0.0):.3f}")
print(f"Avg response time: {results['summary']['avg_response_time']:.2f}s")
print(f"Successful: {results['summary']['successful_queries']}/{results['summary']['total_questions']}")

## 📈 Step 4: Compare Results

**What This Cell Does:**
- Calculates raw statistics (mean, std, confidence intervals)
- Computes improvement metrics (absolute and percentage changes)
- Validates experimental conditions (same questions, documents)
- Saves all data to JSON file

**Notebook Output:**
- ✅ Minimal summary (just confirms it worked)
- ✅ Quick stats (baseline vs SimRAG scores)
- ✅ Download instructions

**Local Display (via `simrag experiment results`):**
- 📊 Formatted tables and comparisons
- 💡 Interpretations ("This is a significant improvement")
- 📈 Statistical significance analysis
- 🎯 Conclusions and recommendations
- ✨ Rich formatting with colors and structure

**Why This Split?**
- Notebook focuses on **computation** (raw statistics)
- Local display focuses on **interpretation** (what it means)
- Easier to read formatted output locally
- Can add insights/recommendations without re-running notebook

In [ ]:
# Compare baseline vs fine-tuned model
# This comparison uses the same statistical methods as the local compare_results.py
# to ensure consistency across all experiments

import math

def calculate_stats(scores_list):
    """
    Calculate statistical measures for context scores
    
    Returns: mean, std, 95% confidence interval
    Used for determining statistical significance of improvements
    
    This matches the statistical analysis in compare_results.py
    """
    all_scores = []
    for scores in scores_list:
        if isinstance(scores, list):
            all_scores.extend(scores)
        elif isinstance(scores, (int, float)):
            all_scores.append(scores)
    
    if not all_scores:
        return {"mean": 0.0, "std": 0.0, "ci_lower": 0.0, "ci_upper": 0.0, "n": 0}
    
    n = len(all_scores)
    mean = sum(all_scores) / n
    variance = sum((x - mean) ** 2 for x in all_scores) / (n - 1) if n > 1 else 0.0
    std = math.sqrt(variance)
    se = std / math.sqrt(n) if n > 0 else 0.0
    margin = 1.96 * se  # 95% confidence interval
    ci_lower = mean - margin
    ci_upper = mean + margin
    
    return {"mean": mean, "std": std, "ci_lower": ci_lower, "ci_upper": ci_upper, "n": n}

baseline_avg = baseline_results["summary"]["avg_context_score"]
simrag_avg = results["summary"]["avg_context_score"]
improvement_percent = ((simrag_avg - baseline_avg) / baseline_avg * 100) if baseline_avg > 0 else 0.0

baseline_stats = calculate_stats(baseline_results["context_scores"])
simrag_stats = calculate_stats(results["context_scores"])

baseline_time = baseline_results["summary"]["avg_response_time"]
simrag_time = results["summary"]["avg_response_time"]

# Create comparison dictionary
# This structure matches compare_results.py for consistency
comparison = {
    "comparison_type": "baseline_vs_simrag",
    "timestamp": datetime.now().isoformat(),
    "validation": {
        # Validate experimental conditions for fair comparison
        "is_valid": True,
        "baseline_questions": len(baseline_results["questions"]),
        "simrag_questions": len(results["questions"]),
        "same_questions": baseline_results["questions"] == results["questions"],  # Must be identical
        "baseline_docs": baseline_results["dataset"]["num_documents"],
        "simrag_docs": results["dataset"]["num_documents"],
        "same_docs": baseline_results["dataset"]["num_documents"] == results["dataset"]["num_documents"]  # Must be identical
    },
    "baseline": {
        "avg_context_score": baseline_avg,
        "context_score_stats": baseline_stats,
        "avg_response_time": baseline_time,
        "avg_answer_quality": baseline_results["summary"].get("avg_overall_score", 0.0),
        "num_questions": len(baseline_results["questions"])
    },
    "simrag": {
        "avg_context_score": simrag_avg,
        "context_score_stats": simrag_stats,
        "avg_response_time": simrag_time,
        "avg_answer_quality": results["summary"].get("avg_overall_score", 0.0),
        "num_questions": len(results["questions"]),
        "model": {
            "stage": STAGE,
            "version": MODEL_VERSION,
            "checkpoint": CHECKPOINT_NAME,
            "base_model": BASE_MODEL
        }
    },
    "improvement": {
        "context_score_improvement": simrag_avg - baseline_avg,
        "context_score_improvement_percent": improvement_percent,
        "answer_quality_improvement": results["summary"].get("avg_overall_score", 0.0) - baseline_results["summary"].get("avg_overall_score", 0.0),
        "response_time_change": simrag_time - baseline_time,
        "response_time_change_percent": ((simrag_time - baseline_time) / baseline_time * 100) if baseline_time > 0 else 0.0,
        "statistical_significance": {
            "baseline_ci": f"[{baseline_stats['ci_lower']:.3f}, {baseline_stats['ci_upper']:.3f}]",
            "simrag_ci": f"[{simrag_stats['ci_lower']:.3f}, {simrag_stats['ci_upper']:.3f}]",
            "overlap": not (baseline_stats['ci_upper'] < simrag_stats['ci_lower'] or simrag_stats['ci_upper'] < baseline_stats['ci_lower']),
            "note": "If CIs don't overlap, difference is statistically significant at p<0.05"
        }
    }
}

# Save comparison results (raw statistics only)
# All formatting and interpretation will be done locally via "simrag experiment results"
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
comparison_file = f"comparison_results_{timestamp}.json"

with open(comparison_file, 'w') as f:
    json.dump(comparison, f, indent=2)

# Minimal output - just confirm it worked
print("\n" + "="*60)
print("COMPARISON COMPLETE")
print("="*60)
print(f"✓ Raw statistics calculated and saved")
print(f"✓ File: {comparison_file}")
print(f"\n📊 Quick Stats:")
print(f"  Baseline context score: {baseline_avg:.3f}")
print(f"  SimRAG context score:   {simrag_avg:.3f}")
print(f"  Improvement:            {improvement_percent:+.1f}%")
print(f"\n📥 Next Steps:")
print(f"1. Download: {comparison_file}")
print(f"2. Place in: comparison_results/ (project root)")
print(f"3. View formatted results: simrag experiment results")
print(f"\n💡 Note: Detailed analysis, interpretations, and conclusions")
print(f"   will be displayed when you run 'simrag experiment results' locally.")

# Download comparison results
files.download(comparison_file)